In [1]:
# from pandas import read_csv
import pandas as pd
filename = '../Week 1/pima-indians-diabetes.data.csv'
names = ['timesPreg', 'plasmaConcenTest', 'bloodPres', 'skinFoldThickness',
         '2HSerumInsulin', 'BMI', 'diabetesPedigree', 'age', 'onsetOfDiabetesbtw5yrs']
# data = read_csv(filename, names=names)
df = pd.read_csv(filename, names=names)
df.head(2) 

,timesPreg,plasmaConcenTest,bloodPres,skinFoldThickness,2HSerumInsulin,BMI,diabetesPedigree,age,onsetOfDiabetesbtw5yrs
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
